In [23]:
#imports
import pandas as pd
import numpy as np
import warnings
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, cohen_kappa_score,mean_squared_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.decomposition import PCA
from pickle import dump,load
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import LinearRegression, ElasticNet


def cohen_kappa_scorer(estimator, X, y):
    y_pred = estimator.predict(X)
    return cohen_kappa_score(y, y_pred)

In [24]:
##### Data #####
data = pd.read_csv(r'C:\Users\annab\OneDrive\STAT 764\Data Sets\flight_data_full.csv')
data = data.drop(['Unnamed: 0','air_time','year', 'month', 'day', 'dest', 'dep_time'],axis=1)


#data for first model
x = data.drop(['dep_delay', 'delay_severity', 'is_delayed'],axis=1)
x = pd.get_dummies(x,dtype=int)
y = data['is_delayed']
x_train1, x_test1, y_train1, y_test1 = train_test_split(x,y,train_size=.7,random_state=764)

In [ ]:
#Best parameters: {'ccp_alpha': 0.0, 'max_depth': None, 'min_impurity_decrease': 0.1, 'min_samples_split': 2}

##rf = RandomForestClassifier(class_weight='balanced',ccp_alpha=0,max_depth=None,min_impurity_decrease=0.1,min_samples_split=2)

##param_grid = {
  ##  'n_estimators': [50, 70, 90, 100, 150, 200, 300],
   ## 'max_features': ['auto', 'sqrt', 'log2', None]
##}

##grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=0, scoring=cohen_kappa_scorer)

##grid_search.fit(x_train1, y_train1)
##best_tree = grid_search.best_estimator_
##print(grid_search.best_score_)
##y_pred = best_tree.predict(x_test1)
##best_params = grid_search.best_params_
##print(f"Best parameters: {best_params}")

In [ ]:
#Best parameters: {'max_features': 'auto', 'n_estimators': 50}, kappa = .457

rf = RandomForestClassifier(class_weight='balanced',ccp_alpha=0,max_depth=None,min_samples_split=2,
                            max_features='sqrt',n_estimators=200)

rf.fit(x_train1, y_train1)
y_pred = rf.predict(x_test1)
print(classification_report(y_test1, y_pred))

y_pred_prob = rf.predict_proba(x_test1)[:, 1]
threshold = 0.15 #.79 both
y_pred_adjusted = (y_pred_prob >= threshold).astype(int)

print("Adjusted Threshold Classification Report:\n", classification_report(y_test1, y_pred_adjusted))


Take his y predictions 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

# Function to preprocess data
def preprocess_data(data):
    # Drop unnecessary columns
    data = data.drop(columns=['Unnamed: 0', 'air_time', 'year', 'month', 'day', 'dest', 'dep_time'])

    # Create target columns
    data['is_delayed'] = data['dep_delay'].apply(lambda x: 1 if x >= 30 else 0)  # On-time vs Delayed
    data['delay_severity'] = data['dep_delay'].apply(lambda x: 1 if 30 <= x <= 120 else (2 if x > 120 else None))  # Minor/Major

    # Drop dep_delay to avoid leakage
    data = data.drop(columns=['dep_delay'])

    # Fill missing values
    data = data.fillna(data.median(numeric_only=True))  # Fill numeric columns with median
    categorical_cols = ['carrier', 'origin', 'type', 'manufacturer', 'model', 'engine']
    for col in categorical_cols:
        data[col] = data[col].fillna(data[col].mode()[0])  # Fill categorical columns with mode

    return data

# Load and preprocess the dataset
data = pd.read_csv(r'C:\Users\annab\OneDrive\STAT 764\Data Sets\flight_data_full.csv')
data = preprocess_data(data)




In [ ]:
# -----------------------------
# First Stage: On-time vs Delayed
# -----------------------------
# Train Random Forest Classifier
rf = RandomForestClassifier(
    class_weight='balanced', ccp_alpha=0, max_depth=None,
    min_samples_split=2, max_features='sqrt', n_estimators=200
)
rf.fit(x_train1, y_train1)

# Predict whether a flight is delayed
y_pred = rf.predict(x_test1)
print("First Stage Classification Report (Default Threshold):\n", classification_report(y_test1, y_pred))

# Adjust threshold for predicting delays
y_pred_prob = rf.predict_proba(x_test1)[:, 1]
threshold = 0.15
y_pred_adjusted = (y_pred_prob >= threshold).astype(int)
print("First Stage Classification Report (Adjusted Threshold):\n", classification_report(y_test1, y_pred_adjusted))

# -----------------------------
# Second Stage: Minor vs Major Delays
# -----------------------------
# Identify delayed flights from the first stage
delayed_indices = (y_pred_adjusted == 1)  # Flights predicted as delayed
X_test_delayed = x_test1[delayed_indices]  # Features of delayed flights
y_test_delayed_actual = data.iloc[X_test_delayed.index]['delay_severity']  # True delay severity

# Prepare data for second stage
data_delayed = data[data['delay_severity'].notna()]  # Only delayed flights with severity labels
X_delayed = data_delayed.drop(columns=['is_delayed', 'delay_severity'])  # Features of delayed flights
y_delayed = data_delayed['delay_severity']  # Target: Minor (1) or Major (2)

# Train-test split for second stage
X_train_delayed, X_test_delayed, y_train_delayed, y_test_delayed = train_test_split(
    X_delayed, y_delayed, test_size=0.3, random_state=42
)

# Train Decision Tree Classifier for Minor vs Major Delays
dt_model = DecisionTreeClassifier(random_state=42, max_depth=5, class_weight='balanced')
dt_model.fit(X_train_delayed.select_dtypes(include=['number']), y_train_delayed)

# Predict and evaluate on second stage
y_pred_dt = dt_model.predict(X_test_delayed.select_dtypes(include=['number']))

print("Second Stage Classification Report (Minor vs Major Delays):")
print(classification_report(y_test_delayed, y_pred_dt, target_names=['Minor Delay', 'Major Delay']))

# Plot confusion matrix for second stage
ConfusionMatrixDisplay.from_estimator(
    dt_model, X_test_delayed.select_dtypes(include=['number']), y_test_delayed,
    display_labels=['Minor Delay', 'Major Delay'], cmap=plt.cm.Blues
)
plt.title("Confusion Matrix: Minor vs Major Delays")
plt.show()

